# Food project

## Discovery (identificazione del problema e degli obiettivi)

Negli ultimi secoli la popolazione mondiale è aumentata notevolmente e secondo l'ONU (fonte del 2019), nel 2050 potrebbe arrivare a quota *10 miliardi*. 

In maniera direttamente proporzionale **sono aumentate anche le domande di cibo, energia ed acqua**, per soddisfare questo bisogno. 

Sfortunamente, i costanti monitoraggi sul cambiamento climatico suggeriscono che il clima della Terra sta cambiando pericolosamente e *parte di questo effetto è direttamente correlato alla produzione agricola e all'allevamento*:

- la produzione alimentare richiede acqua ed energia; 
- la produzione energetica tradizionale richiede risorse idriche; 
- l'agricoltura fornisce una potenziale fonte di energia e tanta acqua. 
 
Garantire che nel mondo tutte le persone abbiano accesso a una dieta nutriente *in modo sostenibile* è una delle maggiori sfide che dobbiamo affrontare. 


> **L'obiettivo di questo notebook** è quello di sfruttare alcuni dati inerenti alla *produzione e distribuzione mondiale di alimenti e mangimi*, per valutarne il loro impatto sull'ambiente in termini di *utilizzo di risorse idriche ed emissioni di gas serra*. 

### Principali spunti e domande che guideranno questa analisi:

1. Il confronto tra produzione di cibo e mangime a livello mondiale

2. Quali paesi sono i maggiori produttori di cibo e mangimi?

3. Quali sono i paesi in maggior crescita in termini di produzione di cibo/consumo? 

4. Sono presenti interessanti outlier nei dati per quanto riguarda la produzione?

5. Quali tipi di alimenti hanno un impatto maggiormente negativo sull'ambiente?

    - Confrontare l'impatto ecologico degli alimenti a base animale
    - Confrontare l'impatto ecologico degli alimenti a base vegetale
    - Confrontare l'impatto ecologico degli alimenti ricchi di proteine


6. Quale fase della produzione alimentare contribuisce maggiormente all'emissione di gas serra?

7. Quali tipi di produzione alimentare dovrebbero essere incoraggiati per il consumo di una dieta maggiormente sostenibile per l'ambiente?

---

## Data selection

Questo progetto sfrutta due datasets caricati su Kaggle: 

- Il [primo](https://www.kaggle.com/datasets/dorbicycle/world-foodfeed-production), fornito dalla FAO (Food and Agriculture Organization of the United Nations), mostra una panoramica della produzione alimentare mondiale dal 1961 al 2013, concentrandosi su un confronto tra alimenti prodotti per il consumo umano e mangimi prodotti per l'allevamento di animali.

- Il [secondo](https://www.kaggle.com/datasets/selfvivek/environment-impact-of-food-production) contiene i valori di utilizzo dell'acqua e le emissioni di gas serra necessari per la realizzazione dei 43 alimenti più comuni prodotti nel mondo.

### Import di dati, moduli e funzioni utili

In [33]:
# Main packages
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Useful packages
from zipfile import ZipFile
from fuzzywuzzy import process
import fuzzywuzzy

pd.set_option('display.max_columns', None)

In [2]:
# Funzioni personalizzate

def zip_extractor():
    """This function looks for zip files and extracts them in the same file path of this script, or in a folder provided 
    with an input. If the folder doesn't exist, it will be created with the provided name."""
    
    destination = input('Where do you want to put the extracted files?\nPress enter if you want to extract files in the current path: ').capitalize()

    if destination == '':
        pass
    elif destination not in os.listdir():
        os.mkdir(destination)

    for dirname, _, filenames in os.walk(os.getcwd()):
        if dirname != 'Zip files':
            for filename in filenames:
                if '.zip' in filename:
                    with ZipFile(filename, "r") as zip:
                        zip.extractall(f'{destination}')

    print("Extraction: Done")
    
    
def csv_nan_reader():
    
    """This function search for .csv files in a target folder (provided by input), skipping 'checkpoint' files 
    and read them in pandas as DataFrame. Then for each file are printed out: 
    
    - filename and parental folder 
    - Dataframe shape 
    - number of total cells 
    - number of cells with missings 
    - % of missing data 
    - rows and columns containing missings 
    - rows and columns in original dataset 
    - rows and columns remained after drop 
    - the effect of dropping rows and columns in terms of remaining data and which is the best method.
    
    If a folder is empty or doesn't exist, this function will segnalate it."""
    
    target_folder = input('Write here folder name: ').capitalize()
    datano = 0

    # Check if the folder exist
    if os.path.isdir(target_folder):
        
        # Check if the folder is not empty
        if os.listdir(target_folder):
            
            for dirname, _, filenames in os.walk(target_folder):
                for filename in filenames:
                    if not 'checkpoint' in filename:
                        name, file_ext = os.path.splitext(filename)
                        
                        if '.csv' in file_ext:
                            
                            path = os.path.join(dirname, filename)
                            df = pd.read_csv(f'{path}')
                            
                            # Extract main information of the file and count Dataframes
                            datano += 1
                            print(f'\n\nDataframe No: {datano}')
                            print('_'*20+'Start'+'_'*20)
                            print(f'This is the Dataset "{filename}" from folder "{dirname}"')
                            print(filename,'shape',df.shape)
                            
                            # How many total missing values do we have?
                            total_cells = np.product(df.shape)
                            print('Total cells:', total_cells)
                            
                            total_missing = df.isnull().sum().sum()
                            print('Total cells with missings:', total_missing)
                            
                            # Count rows containing missing values and check what happens if you drop rows
                            rows_with_missing = df[df.isnull().any(axis=1)].index.to_list()
                            print('\nTotal lenght of missing rows:', len(rows_with_missing))
                            
                            if len(rows_with_missing) > 10:
                                print(f'Rows containing missing values:\n{rows_with_missing[:10]}...[{rows_with_missing[-1]}]')
                            else:
                                print('Rows containing missing values:\n',rows_with_missing)
                                
                            drop_rows = df.dropna()
                            rows_removal_perc = round((1-drop_rows.shape[0]/df.shape[0])*100,2)
                            print(f"\nRows in original dataset: {df.shape[0]}")
                            print(f"Rows remained after drop: {drop_rows.shape[0]}")
                            print('\nDropping rows with NaN removed',rows_removal_perc,'% of the data!')
                            print(f'Shape of Dataframe after rows manipulation: {drop_rows.shape}')
                            
                            # Count columns containing missing values and check what happens if you drop columns
                            cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
                            print('\n\nTotal lenght of missing columns:', len(cols_with_missing))
                            
                            if len(cols_with_missing) > 5:
                                print(f'Columns containing missing values:\n{cols_with_missing[:5]}...[{cols_with_missing[-1]}]')                                
                            else:
                                print('Columns containing missing values:\n',cols_with_missing)
                                
                            drop_cols = df.dropna(axis=1)
                            cols_removal_perc = round((1-drop_cols.shape[1]/df.shape[1])*100,2)
                            print(f"\nColumns in original dataset: {df.shape[1]}")
                            print(f"Columns remained after drop: {drop_cols.shape[1]}")
                            print('\nDropping columns with NaN removed',cols_removal_perc,'% of the data!')
                            print(f'Shape of Dataframe after columns manipulation: {drop_cols.shape}\n\n')

                            # Show info for each col: dtype, unique, %missing
                            print("Dtype, Unique values and Missing(%) of each columns:\n")
                            df_info= pd.DataFrame({"Dtype": df.dtypes, "Unique": df.nunique(),
                            "Missing%": round(df.isnull().sum()/df.shape[0]*100, 2)})
                            print(df_info, '\n')
                            
                            # Compare rows drop vs cols drop and print what is better
                            if rows_removal_perc < cols_removal_perc:
                                print(f"Dropping rows with NaN is the best approach.({rows_removal_perc}% vs {cols_removal_perc}%)\n\n")
                            elif rows_removal_perc > cols_removal_perc:
                                print(f"Dropping columns with NaN is the best approach.({cols_removal_perc}% vs {rows_removal_perc}%)\n\n")
                            else:
                                print("Dropping rows or columns produced the same output.")
                            print('-'*20+'End'+'-'*20+'\n\n')
                            
                        else:
                            print(f"\nI found a file with different extension from csv. '{name}' is a '{file_ext}' file.")
            
            print('\nAll Done.')
        
        else:
            print('\nThis folder is empty!')
    else:
        print("\nSelected folder doesn't exist in this path!")

In [3]:
# Importo i files zip dei due dataset usando le API di Kaggle e li estraggo in una cartella 
# da me designata con una funzione personalizzata

# !kaggle datasets download -d dorbicycle/world-foodfeed-production
# !kaggle datasets download -d selfvivek/environment-impact-of-food-production
# zip_extractor()

In [4]:
# Dataset della produzione di cibi/mangimi dai vari paesi del mondo
fao_df = pd.read_csv('Data/Fao.csv')
df_prod = fao_df.copy()

# Dataset dei consumi di acqua e della produzione di gas serra
consumption_df = pd.read_csv('Data/Food_Production.csv')
df_cons = consumption_df.copy()

## Data cleaning

### Dataset della produzione di cibi/mangimi dei vari paesi del mondo

In [5]:
df_prod.head()

,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
0,AFG,2,Afghanistan,2511,Wheat and products,5142,Food,1000 tonnes,33.94,67.71,1928.0,1904.0,1666.0,1950.0,2001.0,1808.0,2053.0,2045.0,2154.0,1819.0,1963.0,2215.0,2310.0,2335.0,2434.0,2512.0,2282.0,2454.0,2443.0,2129.0,2133.0,2068.0,1994.0,1851.0,1791.0,1683.0,2194.0,1801.0,1754.0,1640.0,1539.0,1582.0,1840.0,1855.0,1853.0,2177.0,2343.0,2407.0,2463.0,2600.0,2668.0,2776.0,3095.0,3249.0,3486.0,3704.0,4164.0,4252.0,4538.0,4605.0,4711.0,4810,4895
1,AFG,2,Afghanistan,2805,Rice (Milled Equivalent),5142,Food,1000 tonnes,33.94,67.71,183.0,183.0,182.0,220.0,220.0,195.0,231.0,235.0,238.0,213.0,205.0,233.0,246.0,246.0,255.0,263.0,235.0,254.0,270.0,259.0,248.0,217.0,217.0,197.0,186.0,200.0,193.0,202.0,191.0,199.0,197.0,249.0,218.0,260.0,319.0,254.0,326.0,347.0,270.0,372.0,411.0,448.0,460.0,419.0,445.0,546.0,455.0,490.0,415.0,442.0,476.0,425,422
2,AFG,2,Afghanistan,2513,Barley and products,5521,Feed,1000 tonnes,33.94,67.71,76.0,76.0,76.0,76.0,76.0,75.0,71.0,72.0,73.0,74.0,71.0,70.0,72.0,76.0,77.0,80.0,60.0,65.0,64.0,64.0,60.0,55.0,53.0,51.0,48.0,46.0,46.0,47.0,46.0,43.0,43.0,40.0,50.0,46.0,41.0,44.0,50.0,48.0,43.0,26.0,29.0,70.0,48.0,58.0,236.0,262.0,263.0,230.0,379.0,315.0,203.0,367,360
3,AFG,2,Afghanistan,2513,Barley and products,5142,Food,1000 tonnes,33.94,67.71,237.0,237.0,237.0,238.0,238.0,237.0,225.0,227.0,230.0,234.0,223.0,219.0,225.0,240.0,244.0,255.0,185.0,203.0,198.0,202.0,189.0,174.0,167.0,160.0,151.0,145.0,145.0,148.0,145.0,135.0,132.0,120.0,155.0,143.0,125.0,138.0,159.0,154.0,141.0,84.0,83.0,122.0,144.0,185.0,43.0,44.0,48.0,62.0,55.0,60.0,72.0,78,89
4,AFG,2,Afghanistan,2514,Maize and products,5521,Feed,1000 tonnes,33.94,67.71,210.0,210.0,214.0,216.0,216.0,216.0,235.0,232.0,236.0,200.0,201.0,216.0,228.0,231.0,234.0,240.0,228.0,234.0,228.0,226.0,210.0,199.0,192.0,182.0,173.0,170.0,154.0,148.0,137.0,144.0,126.0,90.0,141.0,150.0,159.0,108.0,90.0,99.0,72.0,35.0,48.0,89.0,63.0,120.0,208.0,233.0,249.0,247.0,195.0,178.0,191.0,200,200


Procedo verificando la presenza di valori nulli (*NaN*) nel dataset, o altri valori incoerenti. Se effettivaemente presenti, approfondisco l'analisi per sapere quanti sono e in quali colonne sono maggiormente frequenti. Inoltre, proverò a fare un *imputation*, ovvero provare a stabilire il motivo della loro presenza. 

In [6]:
print(f'Dimensioni del dataframe: {df_prod.shape}')

# Quante celle contengono valori nulli rispetto alle celle totali?
total_cells = np.product(df_prod.shape)
print(f'Numero totale di celle: {total_cells}')

total_missing = df_prod.isnull().sum().sum()
print(f'Numero totale di celle con valori nulli: {total_missing}')

percent_missing = round(total_missing/total_cells * 100, 2)
print(f'\nPercentuale di valori nulli in questo Dataframe: {percent_missing}%')

Dimensioni del dataframe: (21477, 63)
Numero totale di celle: 1353051
Numero totale di celle con valori nulli: 117450

Percentuale di valori nulli in questo Dataframe: 8.68%


In [31]:
pd.set_option('display.max_rows', 63)

# Costruisco una tabella riassuntiva sui Dtype, valori unici e valori nulli di tutte le colonne
print("Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:")

df_prod_info= pd.DataFrame({"Dtype": df_prod.dtypes, 
                            "Valori unici": df_prod.nunique(),
                            "Valori mancanti(%)": round(df_prod.isnull().sum()/df_prod.shape[0]*100, 2)
                            }).rename_axis('Colonne', axis='rows')                       

df_prod_info

Dtype, Valori unici e Valori mancanti(%) di ciascuna colonna:


,Dtype,Valori unici,Valori mancanti(%)
Colonne,,,
Area Abbreviation,object,169,0.00
Area Code,int64,174,0.00
Area,object,174,0.00
Item Code,int64,117,0.00
Item,object,115,0.00
Element Code,int64,2,0.00
Element,object,2,0.00
Unit,object,1,0.00
latitude,float64,173,0.00


Si vede chiaramente che la maggior parte dei valori nulli presenti nel dataset è concentrata nelle colonne *tra il 1961 e il 1991*. Come si può leggere tra la documentazione nella pagina Kaggle dove è fornito il dataset:

>The Food Balance sheet's data was relatively complete. A few countries that do not exist anymore, such as Czechoslovakia, were deleted from the database. Countries which were formed lately such as South Sudan were kept, even though they do not have all full data going back to 1961. [...]

Quindi, in un certo senso, sembrerebbe che particolari *evoluzioni geopolitiche di alcuni paesi* siano la causa dell'indisponibilità dei dati.  

Prendo un campione a caso dove sono presenti questi valori nulli, cercando di fare chiarezza e di confermare l'informazione contenute nella documentazione del dataset.  

In [8]:
# Creo una lista con gli indici delle righe che contengono i valori nulli
rows_with_missing = df_prod.loc[df_prod.isnull().any(axis=1)].index.to_list()
print(f'\nNumero di righe con valori nulli: {len(rows_with_missing)}')

# Imposto un random seed per rendere pseudorandomica la riproducibilità di questa cella
np.random.seed(0)

# Utilizzo la lista per creare un dataframe avente solo le righe che contengono valori nulli
df_null_rows = df_prod.loc[rows_with_missing]

# Seleziono un campione di 5 elementi
df_null_rows.sample(5)


Numero di righe con valori nulli: 3539


,Area Abbreviation,Area Code,Area,Item Code,Item,Element Code,Element,Unit,latitude,longitude,Y1961,Y1962,Y1963,Y1964,Y1965,Y1966,Y1967,Y1968,Y1969,Y1970,Y1971,Y1972,Y1973,Y1974,Y1975,Y1976,Y1977,Y1978,Y1979,Y1980,Y1981,Y1982,Y1983,Y1984,Y1985,Y1986,Y1987,Y1988,Y1989,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,Y1996,Y1997,Y1998,Y1999,Y2000,Y2001,Y2002,Y2003,Y2004,Y2005,Y2006,Y2007,Y2008,Y2009,Y2010,Y2011,Y2012,Y2013
17328,SVK,199,Slovakia,2517,Millet and products,5521,Feed,1000 tonnes,48.67,19.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1,1
13020,MNE,273,Montenegro,2617,Apples and products,5142,Food,1000 tonnes,42.71,19.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,19.0,34.0,38.0,33.0,27.0,14,13
17526,SVN,198,Slovenia,2733,Pigmeat,5142,Food,1000 tonnes,46.15,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,81.0,84.0,75.0,70.0,73.0,77.0,87.0,76.0,82.0,78.0,88.0,88.0,88.0,89.0,84.0,84.0,80.0,81.0,76.0,69,58
11128,LVA,119,Latvia,2656,Beer,5142,Food,1000 tonnes,56.88,24.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,88.0,53.0,66.0,67.0,66.0,73.0,82.0,100.0,99.0,105.0,134.0,142.0,124.0,144.0,147.0,158.0,156.0,146.0,164.0,172.0,163,156
16103,RUS,185,Russian Federation,2560,Coconuts - Incl Copra,5142,Food,1000 tonnes,61.52,105.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,2.0,2.0,10.0,10.0,21.0,28.0,42.0,45.0,47.0,52.0,45.0,59.0,42.0,39.0,48.0,63.0,57,63


Possiamo vedere che in questi 5 paesi selezionati a caso ('Slovacchia', 'Montenegro', 'Slovenia', 'Lettonia' e 'Federazione Russa'), in effetti i dati di produzione sono mancanti dal 1961 al 1991 (il Montenegro fino al 2005). 

Cercando informazioni sulla storia geopolitica di questi paesi (fonte: [Wikipedia](https://it.wikipedia.org/wiki/Pagina_principale)) si può leggere:

- *Slovacchia*: [...] La Repubblica Slovacca e la Repubblica Ceca sono nate il 1º gennaio 1993 dalla divisione, sancita dal parlamento della Cecoslovacchia, che **già dal 1990 aveva assunto il nome di Repubblica Federale Ceca e Slovacca**;

- *Montenegro*: [...] Fino al 2 giugno 2006 la Repubblica del Montenegro è stata unita alla Repubblica di Serbia con il nome di Serbia e Montenegro. **Dal 3 giugno 2006 il Montenegro è diventato uno Stato indipendente**;

- *Slovenia*: [...] Dal 1º maggio 2004 la Slovenia è membro dell'Unione europea e la valuta nazionale, dal 1º gennaio 2007, è l'euro, che ha sostituito il tallero sloveno, adottato **nel 1991 dopo l'indipendenza** [...];

- *Lettonia*: [...] L'URSS riconobbe **la Lettonia come Stato indipendente il 6 settembre 1991** [...];

- *Federazione Russa*: [...] Nell'agosto del 1991 il tentato colpo di Stato militare per deporre Gorbačëv e preservare l'Unione Sovietica portò invece alla fine del Partito Comunista dell'Unione Sovietica. Nonostante la contraria volontà espressa dal popolo, **il 26 dicembre 1991 l'Unione Sovietica si dissolse in quindici Stati post-sovietici**.

Questo risultato è in linea con la descrizione riportata dalla documentazione del dataset, giustificando il retroscena che riguarda presenza dei valori mancanti.

Proviamo ora a vedere se **è conveniente eliminare le righe del dataframe che contengono valori nulli**, oppure se si rischia di **perdere un numero eccessivo di informazioni**.  

In [9]:
# Creo un dataframe droppando le righe che contengono valori nulli 
# 'rd' sta per 'row dropped'
df_prod_rd = df_prod.dropna()

# Calcolo la percentuale di dati eliminati rispetto dal dataframe originale
rows_removal_perc = round((1-df_prod_rd.shape[0]/df_prod.shape[0])*100,2)

# Stampo informazioni sul numero di righe totali e shape
print(f"\nRighe del dataframe originale: {df_prod.shape[0]}. Shape del dataframe originale: {df_prod.shape}")
print(f"Righe del dataframe dopo aver rimosso i valori mancanti: {df_prod_rd.shape[0]}. Shape del dataframe modificato: {df_prod_rd.shape}")
print(f'\nLa rimozione delle righe che contengono valori NaN ha eliminato il {rows_removal_perc}% dei dati!')


Righe del dataframe originale: 21477. Shape del dataframe originale: (21477, 63)
Righe del dataframe dopo aver rimosso i valori mancanti: 17938. Shape del dataframe modificato: (17938, 63)

La rimozione delle righe che contengono valori NaN ha eliminato il 16.48% dei dati!


Avrei anche potuto sostituire i valori NaN con 0 tramite *fillna()*. Tuttavia, voglio visualizzare la produttività dei vari paesi nell'arco temporale più lungo possibile. Inoltre secondo me, operando in questo modo **si perdono quantità tollerabili di dati**. Quindi, continuerò lo studio di questo dataset adottando questa procedura.

Procedo ora per controllare eventuali *incongruenze* nei valori delle colonne 'Area' e 'Item', che si riferiscono ai paesi e agli alimenti prodotti.

In [10]:
# Per comodità riporto il nome del dataframe ad uno più semplice
df = df_prod_rd.copy()

In [11]:
# Creo una serie per vedere se e quali etichette di 'Area Abbreviation' sono state utilizzate per più di un paese
find_duplicates = df.groupby('Area Abbreviation')['Area'].nunique()
find_duplicates[find_duplicates > 1]

Area Abbreviation
CHN    4
Name: Area, dtype: int64

In [12]:
# Uso l'etichetta 'CHN' per creare un filtro
chn_filter = df['Area Abbreviation'].isin(['CHN'])

# Uso il filtro per estrapolare un dataframe composto solo da paesi con etichetta CHN
chn_df = df.loc[chn_filter]

# Seleziono la colonna 'Area' e trasformo l'array dei valori unici in una lista
chn_countries = chn_df.Area.unique().tolist()
chn_countries

['China, Hong Kong SAR',
 'China, Macao SAR',
 'China, mainland',
 'China, Taiwan Province of']

In [13]:
# Rinomino tutte le località della Cina sotto un'unica voce usando un dictionary comprehension
df.Area.replace({n:'China' for n in chn_countries}, inplace=True)

Uno strumento utilissimo per controllare ulteriori incongruenze tra tutti gli altri elementi della colonne 'Area' è *fuzzywuzzy*. 

Questo modulo utilizza la distanza di [Levenshtein](https://it.wikipedia.org/wiki/Distanza_di_Levenshtein), per misurare la differenza fra due stringhe e a determinare quindi quanto due stringhe siano simili.

In [ ]:
# TODO riprendi da qui, forse conviene usare fuzzywuzzy direttamente sulla colonna Area!
# [ ] prova solo con lower o con lower + strip

In [20]:
df['Area'] = df.Area.str.lower()
df['Paesi'] = df['Paesi'].str.strip()

In [35]:
df.Paesi = df.Paesi.str.capitalize()

In [37]:
conf = df.Paesi == df.Area
conf.value_counts()

True     13833
False     4105
dtype: int64

In [ ]:
test = df.Paesi.unique().tolist()
test

In [32]:
for n in test:
    matches = fuzzywuzzy.process.extract(n, test, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    if matches[1][1] >= 85:
        print(f'{matches[0:3]}\n')

[('australia', 100), ('austria', 88), ('mauritania', 63)]

[('austria', 100), ('australia', 88), ('costa rica', 59)]

[("democratic people's republic of korea", 100), ("lao people's democratic republic", 87), ('republic of korea', 63)]

[('iceland', 100), ('ireland', 86), ('finland', 71)]

[('ireland', 100), ('iceland', 86), ('finland', 71)]

[("lao people's democratic republic", 100), ("democratic people's republic of korea", 90), ('dominican republic', 56)]



In [ ]:
# TODO aggiungi alla fine della correzione di 'Area' queste parti, prima di procedere con 'Item'

In [ ]:
# Cambiare Republic of Korea in South Korea
# Cambiare Democratic People's Republic of Korea in North Korea
koreani = df.loc[df.Area.str.contains('Korea')]
koreani.Area.value_counts()


In [ ]:
# Aggiustare nome C�te d'Ivoire in Ivory Coast ed eliminare le spaziature (con strip()?)
pd.set_option('display.max_rows', None)
df_prod.Area.value_counts().sort_index()

Termino la procedura di data cleaning per questo dataset sistemando l'etichetta di alcune colonne ed eliminando quelle che non sono utili alla mia analisi. 

In [ ]:
# Rimuovo la 'Y' dalle labels degli anni con un dictionary comprehension
df.rename(columns={n:n[1:] for n in df.columns if 'Y' in n}, inplace=True)

# Con lo stesso metodo di prima rimuovo gli spazi presenti nelle labels
df.rename(columns={n:n.replace(' ', '_') for n in df.columns if ' ' in n}, inplace= True)

# Elimino le colonne non necessarie
df.drop(columns=['Area_Abbreviation', 'Area_Code', 'Element_Code', 'Unit'], inplace=True)

### Dataset dei consumi di acqua e della produzione di gas serra